In [130]:
import numpy as np
import sys
sys.path.append('../just2017plots')
import plotting_resources as res
from astropy.table import Table, vstack, join

rauto_cut = 22.9

In [131]:
slitcat = Table().read('../catalogs/personal_catalogs/slits_phot_zs_cutonslitdist_rcl.csv', format='csv')
speccat = slitcat[np.where((slitcat['slit_distance'] < 1.0) &
                           (slitcat['zSpec'] > 0.0) & 
                           (slitcat['zSpec_Q'] == 1))]
cllist = res.load_clusters()
rfmags = Table().read('../catalogs/eazy_colors/ediscs_zldp_zfors_EAzY_outputs.csv', format='csv')
rfmags.rename_column('id', 'ids')

# Add the relevant Rest Frame colors to the object catalog
MAG_AB_rfB, MAG_AB_rfU

In [132]:
fulltable = join(speccat, rfmags['ids','MAG_AB_rfB', 'MAG_AB_rfU'], keys='ids', join_type='left')
fulltable['UB_color'] = fulltable['MAG_AB_rfU'] - fulltable['MAG_AB_rfB']

# Recast spec_type as an integer, clean up bad things to be 0

In [142]:
print(set(fulltable['spec_type']))

{'1', '0', '3', '4', '2'}


In [150]:
for row in fulltable:
    if len(row['spec_type']) > 1:
        row['spec_type'] = '0'

fulltable['spec_type_int'] = [int(s) for s in fulltable['spec_type']]
cmrspec1 = fulltable[np.where(fulltable['spec_type_int'] == 1)]
cmrspec2 = fulltable[np.where(fulltable['spec_type_int'] == 2)]
cmrspec = vstack((cmrspec1, cmrspec2))

In [151]:
print(cmrspec.colnames)

['field', 'ids', 'ra', 'dec', 'x', 'y', 'B1', 'V1', 'R1', 'I1', 'z1', 'B2', 'V2', 'R2', 'I2', 'z2', 'B3', 'V3', 'R3', 'I3', 'z3', 'Bauto', 'Vauto', 'Rauto', 'Iauto', 'zauto', 'zLDP5_7', 'Q5_7', 'zphot', 'zphot_errUP', 'zphot_errLO', 'class_StarR', 'zLDP', 'zLDPerr', 'Q', 'zLDP_good', 'slit_distance', 'zSpec', 'zSpec_Q', 'spec_type', 'theta_cl_radian', 'd_cl_mpc', 'MAG_AB_rfB', 'MAG_AB_rfU', 'UB_color', 'spec_type_int']


# Loop over clusters and find the CMR using the average U-B of members

In [154]:
for cl in cllist:
    #objs = cmrspec[np.where((cmrspec['field'] == cl['tablename']) &
    #                        (abs(cmrspec['zSpec']-cl['z']) < 0.02))]
    objs = cmrspec[np.where((abs(cmrspec['zSpec']-cl['z']) < 0.02))]
    cl['cmr'] = np.mean(objs['UB_color'])
    print(len(objs))

with open('personal_catalogs/cluster_cmr.dat', 'w') as fout:
    fout.write('papername, cmr\n')
    for cl in cllist:
        fout.write('{}, {}\n'.format(cl['papername'], cl['cmr']))

24
8
11
15
14
10
18
11
15
24
16
9
8
12
22
5
8
10
6
11
16
